In [80]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve
from scipy.optimize import minimize

In [270]:
def FOC(kj,ki,gammai,Vj,t,r,p):

    # The 'x' vector to be optimized is the vector of physician strategies, kj

    # Patient strategies
    Ui_aux = Vj*ki.reshape((-1,1)) - t     # Vj x ki
    ki_greater = np.array(ki.reshape((-1,1)) >= kj.reshape((1,-1))).astype(int)  # I x J boolean mask of ki > kj
    Ui = Ui_aux + ki_greater * gammai.reshape((-1,1))   # Vj x ki + gammai if ki > kj
    exp_Ui = np.exp(Ui)     # exp(Ui)
    αi = np.zeros((I,J))
    αi[Ui > 0] = exp_Ui[Ui > 0]
    sum_αi = np.sum(αi, axis = 1).reshape((-1,1))
    Si = np.divide(αi, sum_αi, where=sum_αi!=0, out=np.zeros_like(αi))   # I x J vector of patient strategies (logit)

    # Aggregates
    Qi = np.sum(Si, axis = 0)
    Xi = np.sum(ki_greater * Si, axis = 0)

    return r - p * Xi


In [308]:
# Number of doctors and patients
I = 10000
J = 500

# Doctor parameters
Vj = np.random.random(J)
r =0.1
p = 100000*np.random.random(J)
t= 0.2

# Patient parameters
ki  = np.random.random(I)
gammai  = np.random.random(I)

# Doctor strategies guess
kj  = 0.5*np.ones(J)

a = FOC(kj,ki,gammai,Vj,t,r,p)

a

array([ -737555.91984602,  -826543.38124738,  -103725.30004021,
        -207767.61463196,  -548768.92721676,  -172544.3850604 ,
        -138307.23057221,  -470980.10515014,    -6972.18342401,
        -112436.15924235,  -246228.29448704,  -243650.39877421,
        -128796.66826781, -1196395.41101902,   -58594.86391908,
        -183266.58923467,  -427354.65416187, -1137574.16342309,
       -1191869.78233662,  -924829.58714185,  -518059.20607317,
        -650718.94483661,   -40861.95354816,  -208546.76554122,
        -843228.85276687,  -855650.23396778,  -998637.56628844,
         -30108.66024673,  -300608.38689727,  -114559.92543644,
        -480343.44875729,  -602175.24177746,   -14181.41480239,
        -598225.05015954,  -425447.75317824,  -127371.19272238,
        -324257.31369898,  -450235.90061523,  -666473.45007301,
        -588104.94470517, -1297088.43601263,  -127421.33855348,
        -758186.72169887,  -235358.19533554,  -572430.69295616,
        -487144.27248536,  -743510.51434

In [353]:
def newton(f, Df, x_0, tol=1e-7, max_iter=100_000):
    x = x_0

    # Implement the zero-finding formula
    def q(x):
        Df_x = Df(x)  # Compute Df(x) once to avoid redundant calculations
        return np.where(Df_x == 0, 0, x - f(x) / Df_x)

    error = tol + 1
    n = 0
    while np.any(error > tol):
        n += 1
        if(n > max_iter):
            break
        y = q(x)
        error = np.where(y == 0, 0, np.abs(x - y))
        
        x = y
        x[x<0] = 0
        if all(x == 0):
            break
    return x, error


h = 0.001
f = lambda x: FOC(x, ki,gammai,Vj,t,r,p)
Df = lambda x: (f(x + h) - f(x))*(1/h)
k_guess = 0.5*np.ones(J)

a, b = newton(f, Df, k_guess, tol=1e-3, max_iter=5)

/var/folders/d9/f39ks_q941961hvlpvd2zymc0000gn/T/ipykernel_46630/2997313409.py:7: RuntimeWarning: divide by zero encountered in divide
  return np.where(Df_x == 0, 0, x - f(x) / Df_x)


In [354]:
a

array([0.        , 1.42542477, 0.        , 0.        , 0.        ,
       0.99037686, 0.        , 0.        , 1.05649219, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.57694542, 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.06013107, 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.20623665,
       0.        , 1.17228998, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.9998754 , 1.22166956, 0.        ,
       1.74873165, 0.        , 1.59327883, 0.        , 1.34988593,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.99816976, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.42503873, 0.        , 0.        , 0.        , 1.14081489,
       0.        , 1.15452511, 1.32907893, 0.        , 0.     

array([     0.        ,      0.        ,      0.        ,      0.        ,
       198859.98855163])

In [282]:
a

array([0., 0., 0., 0., 0.])

array([0.1, 0.1, 0.1, 0.1, 0.1])